## Import Packages and Reading of Data

In [1]:
import re
import demoji
import emoji
import warnings
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import skew
from datetime import datetime
from datetime import date
from collections import OrderedDict
from sklearn.base import BaseEstimator, TransformerMixin
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [2]:
df_metadata = pd.read_csv("final_combined_dataset.csv")

In [3]:
df_metadata

Unnamed: 0          id              name        username  \
0               0    18042464      [日向の乙女]ミルミクス          MILMIX   
1               1    60286377    Tahnee Trotter   tahneetrotter   
2               2   373791732  Kassandra Garcia   Sassy_Kassy37   
3               3  2374895658      TonieSabella    TonieSabella   
4               4    17537004    Mihai Todor 🇺🇦      MihaiTodor   
...           ...         ...               ...             ...   
20284       20289    15367428      Paul Urmston     paulurmston   
20285       20290    17006395    Laura Zambelli   laura_trouble   
20286       20291    22130752  The Stag Company  TheStagCompany   
20287       20292    15166188      kelsey lee 🖤     kelseyofzen   
20288       20293  1770131138           𑁍𝚌𝚑𝚎𝚕𝚊𑁍   nosnaheislehc   

                     created_at  protected  verified                      url  \
0      2008-12-11T06:18:59.000Z      False     False  https://t.co/AoobuYGh5y   
1      2009-07-26T11:25:30.000Z      False     False   http://t.co/P7D9W9j5nT   
2      2011-09-15T05:49:37.000Z      False     False                      NaN   
3      2014-03-06T06:01:31.000Z      False     False   http://t.co/yakUEMw5xd   
4      2008-11-21T14:28:51.000Z      False     False  https://t.co/Ab2jd2vhvV   
...                         ...        ...       ...                      ...   
20284  2008-07-09T16:38:23.000Z      False     False                      NaN   
20285  2008-10-27T20:27:57.000Z      False     False  https://t.co/lFTff06wEg   
20286  2009-02-27T11:31:33.000Z      False     False   http://t.co/GiEeJEGx48   
20287  2008-06-19T04:57:53.000Z      False     False                      NaN   
20288  2013-09-07T19:41:40.000Z      False     False  https://t.co/3bIqCXqI6Q   

                                       profile_image_url         location  \
0      https://pbs.twimg.com/profile_images/788469686...               日本   
1      https://pbs.twimg.com/profile_images/488235175...              NaN   
2      https://pbs.twimg.com/profile_images/290477377...              NaN   
3      https://pbs.twimg.com/profile_images/473894199...         Bulgaria   
4      https://pbs.twimg.com/profile_images/781079151...  Dublin, Ireland   
...                                                  ...              ...   
20284  https://pbs.twimg.com/profile_images/161391553...   United Kingdom   
20285  https://pbs.twimg.com/profile_images/126498263...              NaN   
20286  https://pbs.twimg.com/profile_images/793413210...     Brighton, UK   
20287  https://pbs.twimg.com/profile_images/147715462...              NaN   
20288  https://pbs.twimg.com/profile_images/126725875...                🐝   

                                             description  followers_count  \
0      テイルズオブシリーズが好きだったり。モバマス貧弱一般人。Splatoon中毒患者。最近はFN...             1959   
1                                            Get frucked              371   
2      If you have Motivatoin, Dedication and Faith i...               36   
3        Ambient/Post Rock music created by Ben Leopard.               30   
4      Principal Software Engineer interested in comp...             1089   
...                                                  ...              ...   
20284                                                NaN               72   
20285  You heard that i was trouble but you couldn't ...              314   
20286  #Stagweekends, #stagdos and #stagnights. We do...             2981   
20287                                    cap ou pas cap?              170   
20288                               she's in a bad way !              971   

       following_count  tweet_count  listed_count account_type  \
0                 2708       405158           157        human   
1                    0        33172            10        human   
2                    0          548             0          bot   
3                  148           93             1          bot   
4                 2283         9098            34

## ETL

In [4]:
def url(df):
    df["has_url"] = ~df["url"].isna()
    return df

def username(df):
    special_char = re.compile('[@_!#$%^&*()<>?/\|}{~:]')
    df["un_no_of_char"] = df["username"].apply(lambda x: len(str(x)))
    df["un_special_char"] = df["username"].apply(lambda x: special_char.search(str(x)) != None)
    return df

def name(df):
    special_char = re.compile('[@_!#$%^&*()<>?/\|}{~:]')
    df["name_no_of_char"] = df["name"].apply(lambda x: len(str(x)))
    df["name_special_char"] = df["name"].apply(lambda x: special_char.search(str(x)) != None)
    return df

def description(df):
    user_tags = r'\B@\w*[a-zA-Z]*\w*'
    hashtags = r'\B#\w*[a-zA-Z]+\w*'
    links = r'(https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}[-a-zA-Z0-9()@:%_+.~#?&/=]*)'
    df["des_no_of_usertags"] = df["description"].apply(lambda x: len(re.findall(user_tags, str(x)))) #str(x).count('@'))
    df["des_no_of_hashtags"] = df["description"].apply(lambda x: len(re.findall(hashtags, str(x)))) #str(x).count('#'))
    df["des_external_links"] = df["description"].apply(lambda x: re.findall(links, str(x)) != [])
    df["has_description"] = ~df["description"].isna() 
    return df

def location(df):
    df["location"] = ~df["location"].isna() # false = location is NaN; true = has location
    return df
  
def time(df):
    # df["created_time"] = df["created_at"].apply(lambda x : datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f%z').date())
    # d1 = date(2022, 10, 8) # date we extracted these data 
    df["account_age_in_days"] = df["created_at"].apply(lambda x : (date(2022, 10, 8) - datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f%z').date()).days)
    df["average_tweets_per_day"] = df["tweet_count"]/df["account_age_in_days"]
    return df

def follow_count(df):
    df["followers_following_count"] = df["followers_count"] * df["following_count"]
    return df

def account_type(df):
    df["isBot"] = df["account_type"].apply(lambda x : 1 if x == 'bot' else 0)
    return df

def convert_PV(variable):
    if variable == 'True':
        return True
    if variable == 'False':
        return False
    if variable == '0.0':
        return False
    if variable == '1.0':
        return True
    return variable

def convert_TF(variable):
    var = 1 if variable == True else 0
    return var

In [5]:
df_metadata = account_type(df_metadata)
df_metadata = username(df_metadata)
df_metadata = name(df_metadata)
df_metadata = description(df_metadata)
df_metadata = location(df_metadata)
df_metadata = url(df_metadata)
df_metadata = time(df_metadata)
df_metadata = follow_count(df_metadata)

pv_var = ["verified", "protected"]

for var in pv_var:
    df_metadata[var] = df_metadata[var].apply(convert_PV)

boolean_var = ["verified", "location", "un_special_char", 
               "name_special_char", "des_external_links", 
               "has_description", "protected", 
               "has_url", "has_profile_image"]

for var in boolean_var:
    df_metadata[var] = df_metadata[var].apply(convert_TF)
    
int_var = ["followers_count", "following_count", "tweet_count", "listed_count"]

for var in int_var:
    df_metadata[var] = df_metadata[var].astype(int)

In [6]:
new_df_metatdata = df_metadata.drop(columns=["Unnamed: 0", "url", "id", "created_at", "name", "username", "description", "profile_image_url", "account_type"])

In [7]:
new_df_metatdata

protected  verified  location  followers_count  following_count  \
0              0         0         1             1959             2708   
1              0         0         0              371                0   
2              0         0         0               36                0   
3              0         0         1               30              148   
4              0         0         1             1089             2283   
...          ...       ...       ...              ...              ...   
20284          0         0         1               72               76   
20285          0         0         0              314              107   
20286          0         0         1             2981             2523   
20287          0         0         0              170              180   
20288          0         0         1              971              319   

       tweet_count  listed_count  has_profile_image  isBot  un_no_of_char  \
0           405158           157                  1      0              6   
1            33172            10                  1      0             13   
2              548             0                  1      1             13   
3               93             1                  1      1             12   
4             9098            34                  1      0             10   
...            ...           ...                ...    ...            ...   
20284         4161             1                  1      0             11   
20285        20915             9                  1      0             13   
20286         4991            46                  1      0             14   
20287         7820             4                  1      0             11   
20288        64923            21                  1      0             13   

       un_special_char  name_no_of_char  name_special_char  \
0                    0               12                  0   
1                    0               14                  0   
2                    1               16                  0   
3                    0               12                  0   
4                    0               14                  0   
...                ...              ...                ...   
20284                0               12                  0   
20285                1               14                  0   
20286                0               16                  0   
20287                0               12                  0   
20288                0                7                  0   

       des_no_of_usertags  des_no_of_hashtags  des_external_links  \
0                       0                   0                   0   
1                       0                   0                   0   
2                       0                   0                   0   
3                       0                   0                   0   
4                       0                   0                   1   
...                   ...                 ...                 ...   
20284                   0                   0                   0   
20285                   0                   0                   1   
20286                   0                   3                   1   
20287                   0                   0                   0   
20288                   0                   0                   0   

       has_description  has_url  account_age_in_days  average_tweets_per_day  \
0                    1        1                 5049               80.245197   
1                    1        1                 4822                6.879303   
2                    1        0                 4041                0.135610   
3                    1        1                 3138                0.029637   
4                    1        1                 5069                1.794831   
...                ...      ...                  ...                     ...   
20284                0        0                 5204       

In [8]:
new_df_metatdata.describe()

protected      verified      location  followers_count  \
count  20289.000000  20289.000000  20289.000000     2.028900e+04   
mean       0.121839      0.052787      0.669575     1.644475e+05   
std        0.327108      0.223614      0.470378     2.742200e+06   
min        0.000000      0.000000      0.000000     0.000000e+00   
25%        0.000000      0.000000      0.000000     7.500000e+01   
50%        0.000000      0.000000      1.000000     3.720000e+02   
75%        0.000000      0.000000      1.000000     1.381000e+03   
max        1.000000      1.000000      1.000000     1.331757e+08   

       following_count   tweet_count   listed_count  has_profile_image  \
count     2.028900e+04  2.028900e+04   20289.000000       20289.000000   
mean      3.901567e+03  3.554824e+04     408.099068           0.975307   
std       4.583297e+04  9.177157e+04    6016.241166           0.155192   
min       0.000000e+00  0.000000e+00       0.000000           0.000000   
25%       1.470000e+02  5.380000e+02       1.000000           1.000000   
50%       3.210000e+02  9.133000e+03       6.000000           1.000000   
75%       9.240000e+02  3.270600e+04      36.000000           1.000000   
max       4.040264e+06  2.778611e+06  533679.000000           1.000000   

              isBot  un_no_of_char  un_special_char  name_no_of_char  \
count  20289.000000   20289.000000     20289.000000     20289.000000   
mean       0.276012      10.738035         0.142343        11.472966   
std        0.447034       2.704169         0.349410         5.468766   
min        0.000000       2.000000         0.000000         1.000000   
25%        0.000000       9.000000         0.000000         8.000000   
50%        0.000000      11.000000         0.000000        12.000000   
75%        1.000000      13.000000         0.000000        14.000000   
max        1.000000      15.000000         1.000000        50.000000   

       name_special_char  des_no_of_usertags  des_no_of_hashtags  \
count       20289.000000        20289.000000        20289.000000   
mean            0.044803            0.159545            0.214599   
std             0.206875            0.606166            0.938193   
min             0.000000            0.000000            0.000000   
25%             0.000000            0.000000            0.000000   
50%             0.000000            0.000000            0.000000   
75%             0.000000            0.000000            0.000000   
max             1.000000            9.000000           19.000000   

       des_external_links  has_description       has_url  account_age_in_days  \
count        20289.000000      20289.00000  20289.000000         20289.000000   
mean             0.058258          0.84430      0.524126          4431.379762   
std              0.234237          0.36258      0.499430           737.401272   
min              0.000000          0.00000      0.000000          2438.000000   
25%              0.000000          1.00000      0.000000          3854.000000   
50%              0.000000          1.00000      1.000000          4782.000000   
75%              0.000000          1.00000      1.000000          4924.000000   
max              1.000000          1.00000      1.000000          5931.000000   

       average_tweets_per_day  followers_following_count  
count            20289.000000               2.028900e+04  
mean                 7.652910               9.765040e+09  
std                 20.561953               6.013095e+11  
min                  0.000000               0.000000e+00  
25%                  0.115400               5.920000e+03  
50%                  1.944585               1.159020e+05  
75%                  7.007623               1.152480e+06  
max                819.407549               7.508444e+13

In [9]:
new_df_metatdata.to_csv("pre_metadata_dataset.csv", index = False)